In [1]:
import sys
import os
from pathlib import Path

project_root = Path(os.getcwd()).parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
from summarization import FlanT5Trainer

c:\Users\nanda\miniconda3\envs\sona-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
CUDA device: NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
trainer = FlanT5Trainer(config_name="flan-t5")
trainer.prepare_data_and_trainer(train_size=100, val_size=10, test_size=10)

Using pretrained LoRA adapters from d:\Jobs\sona-ai\cp\flan-t5-small-mediasum...
trainable params: 1,376,256 || all params: 78,337,408 || trainable%: 1.7568
Preparing dataset...
Loading processed dataset from disk...


d:\Jobs\sona-ai\summarization\FlanT5Trainer.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  return Seq2SeqTrainer(


In [4]:
trainer.train()

Starting LoRA fine-tuning...
Using device: mps
Training size: 100 | Val size: 10 | Test size: 10


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.715700,4.037961,0.113544,0.038249,0.079052,0.079409
2,3.622300,4.033806,0.113544,0.038249,0.079052,0.079409
3,3.636800,4.031045,0.110849,0.036198,0.076937,0.076450
4,4.071600,4.029676,0.110849,0.036198,0.076937,0.076450


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Saving LoRA adapters to /Users/komangwikananda/Documents/Work/Sona AI/sona-ai/cp/flan-t5-small-mediasum


In [4]:
trainer.test()

Using device: cuda
Testing model on 10 samples...
Evaluating LoRA model...


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Evaluating Base model (no LoRA)...



--- Sample Predictions Comparison ---
Target: Bill Nye Tweets about Global Warming's Effects on Texas Floods
Base  : Bill Nye the Science Guy starting a storm of his own, posting this tweet chastising meteorologists for refusing to tell people that climate change plays a major role in the flooding across Texas. On Thursday the president added his voice warning Americans to be better prepared. BARACK OBAMA, PRESIDENT OF THE UNITED STATES: The best climate scientists in the world ar telling us that extreme weather events, like hurricanes, are likely to become more powerful. When you combine stronger storms with rising seas, that's a recipe for more devastating floods. Climate change didn't cause Hurricane Sandy but it might have made it stronger. On Thursday the president added his voice warning Americans to be better prepared. BARACK OBAMA, PRESIDENT OF THE UNITED STATES: The best climate scientists in the world ar telling us that extreme weather events, like hurricanes, are likely to 

{'metrics': {'lora_metrics': {'test_loss': 3.6296546459198,
   'test_model_preparation_time': 0.0101,
   'test_rouge1': 0.17877353764655649,
   'test_rouge2': 0.06075168720745394,
   'test_rougeL': 0.12455872278805033,
   'test_rougeLsum': 0.12776967262633582,
   'test_runtime': 24.1539,
   'test_samples_per_second': 0.414,
   'test_steps_per_second': 0.124},
  'base_metrics': {'test_loss': 3.63342022895813,
   'test_model_preparation_time': 0.0101,
   'test_rouge1': 0.15439850506816252,
   'test_rouge2': 0.05241204749206198,
   'test_rougeL': 0.10582000138180044,
   'test_rougeLsum': 0.10785800212892474,
   'test_runtime': 22.0333,
   'test_samples_per_second': 0.454,
   'test_steps_per_second': 0.136}},
 'lora_predictions': ["Bill Nye the Science Guy starting a storm of his own, posting this tweet chastising meteorologists for refusing to tell people that climate change plays a major role in the flooding across Texas. On Thursday the president added his voice warning Americans to be 

# Inference

In [2]:
from summarization import FlanT5Inferencer

c:\Users\nanda\miniconda3\envs\sona-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
CUDA device: NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
text = f"Nik: I don't think we should apply this change. \nRen: I agree. The helmet make the side ears looks too small. The face proportion looks weird. \nNik: But, I like the helmet tho. It looks kinda nice. \nRen: Should I redraw and adjust this? So we can keep it. \nNik: Yeah, I really think wearing a helmet will make this character looks awesome. It is just this helmet is not suitable for him. But, the design is cool tho. Maybe for the other character. \nRen: Yeah, sure. I will redraw it, then I will comeback to you later. \nNik: Ok, sure."
print(text)

Nik: I don't think we should apply this change. 
Ren: I agree. The helmet make the side ears looks too small. The face proportion looks weird. 
Nik: But, I like the helmet tho. It looks kinda nice. 
Ren: Should I redraw and adjust this? So we can keep it. 
Nik: Yeah, I really think wearing a helmet will make this character looks awesome. It is just this helmet is not suitable for him. But, the design is cool tho. Maybe for the other character. 
Ren: Yeah, sure. I will redraw it, then I will comeback to you later. 
Nik: Ok, sure.


In [5]:
inferencer = FlanT5Inferencer(
    config_name="flan-t5",
    use_pretrained=True,
    device="cpu",
    max_new_tokens=256,
    num_beams=4,
)
summary = inferencer.generate(text)
print(summary)

Using pretrained LoRA adapters from d:\Jobs\sona-ai\cp\flan-t5-small-mediasum...
Nik and Ren will redraw the helmet.
